# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** It is a classification type of supervised leanring problem to model the factors that predict how likely a student is to pass their high school final exam, by constructing an intervention system.

Difference between Classification and Regression is whether it focus on caterogizing out of data or predicting a output of continous value using variables as predictors. Supervised classification tries to find boundary, which tends to be discrete ouput such as  pass/not pass, wheras in regression it's whole other thing, we're try to find the trend of the data, linear/curve line that we can find to best find the trend of the data. 

So in this case, It is classification type since output type is discrete(class label such as pass/not pass) and what we are trying to find is decision boundary.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [6]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = n_students - n_passed

print type(n_students)
n_students = float(n_students)  #it's python int type so, just convert float()
print type(n_students)

# TODO: Calculate graduation rate
grad_rate = n_passed/n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "*"*60
print "** updated Number of features ** "
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

<type 'int'>
<type 'float'>
Total number of students: 395.0
************************************************************
** updated Number of features ** 
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [7]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [8]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [9]:
# TODO: Import any additional functionality you may need here
#from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# #shufflesplit
# n_iter=50
# ss = ShuffleSplit(len(y_all), n_iter=n_iter, test_size=95)
# # # TODO: Shuffle and split the dataset into the number of training and testing points above
# # #Just select  randomly X_all and named it as  X_feature, and y_all to y_feature 
# # X_feature, X_zero, y_label, y_zero = train_test_split(X_all, y_all, test_size=0.0, random_state=42)

# # print len(X_feature), len(X_zero), len(y_label), len(y_zero)
# # #and then split it into 300 trains, 95 tests 
# X_train = X_train[:num_train]
# X_test = X_test[num_train:]
# y_train = y_train[:num_train]
# y_test = y_test[num_train:]

#X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=42)

### Reflect unbalanced nature of our data set using Stratified K-Fold and Stratified Shuffle Split Cross validation ###
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, test_size=95, random_state=42)
print len(X_train),len(X_test),len(y_train),len(y_test)

# X_all = X_all.reindex(np.random.permutation(X_all.index))
# y_all = y_all.reindex(np.random.permutation(y_all.index))
                      
# X_train = X_all[:num_train]
# X_test = X_all[num_train:]
# y_train = y_all[:num_train]
# y_test = y_all[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

300 95 300 95
Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **Three suprevised learning models :
*  Naive Bayes : 
  * strength : The relative simplicity of the algorithm and the independent features assumption of   Naive Bayes make it a strong performer for classifying texts
  * weakness : it matters what's the relation of the features in one data samples, it only calculate the accumulate
    probabilities and ignore the relationships of the features.
  * General application : text classifying
  * why choose : simple algorithm to deploy.

*  Decision Tree: 
  * strength of the Decision Tree compare to SVM, is because it's really fast, robust to noise and missing values,accurate
    easy to use, computationally cheap.  robust to noise and missing data. 
  * weakness is a problem has lots of parameters and prone to overfitting, so  may want to stop the growth of tree at the appropriate time. Complex trees are hard to interpret. Duplication within the same sub-tree.
  * General application : medical diagnosis, cridit risk analysis.
  * why choose : fast run time. computationally cheap


*  Service Vector Machine: SVM choose the line that maximize the distance of both nearest point. This distance often called Margin.SVM tries to maximize the margin.
 * strength :
    * If we can see clear distance separable plane between two datasets, then it's a good idea to use SVM.
    * Training is relatively easy • No local optimal
    * It scales relatively well to high dimensional data – Tradeoff between classifier complexity and error can be controlled explicitly 
    * Non-traditional data like strings and trees can be used as input to SVM, instead of feature vectors
 * weakness :
    * Because SVM has firm belief(shown by 'margin'), it has weakness in datasets that have close margin.It also tends to not work so well given huge dataset and huge parameters. It still good, but the algorithm will be very slow . Need to select a good kenel function. Model paremeters are difficult to interpret. Requires significant memory and processing prower.

 * General application :  Text classification , Image classification, Handwirting recognition
 * why choose : Data point for this project is as many as <span style="color:red">30 features </span>but SVM is relatively well to high dimension data.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
# Todo something since woring with X_train, y_train ...!!
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()

    # Print and return results
    # The F1 score(test accuracy) can be interpreted as a weighted average of the precision and recall,
    # where an F1 score reaches its best value at 1 and worst at 0:
    # F1 = 2 * (precision * recall) / (precision + recall)
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [12]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=42)
clf_B = GaussianNB()
clf_C = SVC(random_state=42)

# # TODO: Set up the training set sizes
# X_train_100 = X_train[:100]
# y_train_100 = y_train[:100]

# X_train_200 = X_train[:200]
# y_train_200 = y_train[:200]

# X_train_300 = X_train[:300]
# y_train_300 = y_train[:300]

# #print len(X_train_100), len(X_train_300)
# # TODO: Execute the 'train_predict' function for each classifier and each training set size

# train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
# train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
# train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
# train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
# train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

## updated : loop
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)
print "*"*50
print len(X_train), len(X_train_100)


DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6452.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7258.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0070 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6838.

GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.7752.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6457.
Tr

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Dicision tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020           |      0.0000            |     1.0000       |    0.6452       |
| 200               |        0.0040           |      0.0020            |     1.0000       |    0.7258       |
| 300               |        0.0070           |      0.0010            |     1.0000       |    0.6838       |

** Classifer 2 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0030            |       0.0010           |       0.7752     |     0.6457      |
| 200               |       0.0030            |       0.0010           |       0.8060     |     0.7218      |
| 300               |       0.0030            |       0.0010           |       0.8134     |     0.7761      |

** Classifer 3 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0040            |       0.0020           |       0.8354     |     0.8025      |
| 200               |       0.0100            |       0.0040           |       0.8431     |     0.8105      |
| 300               |       0.0170            |       0.0050           |       0.8664     |     0.8052      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
GaussianNB is generally the most appropriate model for 300 training set based on actual student number 395(300 for train, 95 for test) since it is the fastest in training time(0.0030), prediction for test time(0.0010) all together, which is better than SVM( training : 0.0170, predictions : 0.0050) and dicision tree(training :0.0070, predictions: 0.0010), also it has relatively good accuracy for predictions such as F1 score 0.7761 better than dicision tree(0.6838) but just less than SVM(0.8052).
It is simple and strong performer for many input features of<span style="color:red"> 30</span> and small size of data points of 395 students.

But If your score card in your organization is to make better accuracy for prediction, there is option "B", which cost you more computation cost for tuning up the algorithm, Support Vector Machine, for your reference, I attach as belows.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
  In Support Vector Machine(SVM) machine learning algorithm, we plot each student data item as a point in 30-dimensional space (30 features we have, such as school, sex, age, address, etc) with the value of each feature being the value of a particular coordinate, and then “learned” by splitting the dataset set into subsets during training time such as classifying into two classes of clouds by finding the sheet of paper suspended in between two clouds of points such that the the sheet of paper's distance from each possible point in a cloud is maximized and new points are correctlry classifed by maximizing the margin corresponds to trying to place this sheet of paper as dead center between the two clouds of points as possible.

And we can find the subbset of either "passed" or not "passed" predictions by SVM "similarity measure" to determine whether two studens are similar to one another or not which in turn help us decide to which cloud of points they belong.



### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [37]:
# from sklearn.cross_validation import cross_val_score,ShuffleSplit

In [28]:
from sklearn.grid_search import GridSearchCV 
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, pos_label="yes")
parameters = { 'C' : [ 0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma' : [ 0.0001, 0.001, 0.1, 10, 100,1000 ] } # Some SVC parameters
ssscv = StratifiedShuffleSplit( y_train, n_iter=10, test_size=0.1) # 1. Let's build a stratified shuffle object
grid = GridSearchCV( SVC(), parameters, cv = ssscv , scoring=f1_scorer) # 2. Let's now we pass the object and the parameters to grid search
grid.fit( X_train, y_train ) # 3. Let's fit it
best = grid.best_estimator_ # 4. Let's reteieve the best estimator found
print best
y_pred = best.predict( X_test ) # 5. Let's make predictions!
print "F1 score: {}".format( f1_score( y_test, y_pred, pos_label = 'yes' ))
print "Best params: {}".format( grid.best_params_ )

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
F1 score: 0.825806451613
Best params: {'C': 1, 'gamma': 0.1}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
I recommend GaussianNB for this project, which calculate a Gaussian conditional probability based on the data and no need for parameter tuning. But for your reference, I present option "B", Service Vector Machine(SVM) tunining up result as follows.

Service Vector Machine, Tuned model has a F1 score of 0.825806451613 with Best params: {'C': 1, 'gamma': 0.1} , which is far better than 0.8052, testing F1 score of untuned SVM.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.